In [1]:
import os
import sys
from time import sleep
from uuid import uuid4

import GCode
import GRBL
import numpy as np

# GCode Playback

# Code:

In [2]:
cnc = GRBL.GRBL(port="/dev/cnc_3018")
cnc.reset()
sleep(0.5)
cnc.laser_mode = 1
sleep(0.5)
print("Laser Mode: {}".format(cnc.laser_mode))

ok
Laser Mode: 1.0


In [3]:
cnc.status

'<Idle|MPos:-184.726,-66.349,-1.000|Bf:15,127|FS:0,0|WCO:0.000,0.000,0.000>'

In [4]:
if "Alarm" in cnc.status:
    cnc.home()

In [14]:
def init(**kwargs):
    program = GCode.GCode(**kwargs)
    program.G0(F=500)
    program.G1(F=300)
    program.M3(S=1) # Track the laser
    program.G28()
    program.G21()  # Metric Units
    program.G90()  # Absolute positioning.
    program.G92(X=0, Y=0, Z=0)
    program.M5()
    return program


def end():
    program = GCode.GCode()
    program.M5()
    program.G28()
    return program


In [13]:
prog = GCode.GCode()
prog.load("/tmp/output_0001.gcode")
full_prog = GCode.GCode(machine=cnc)
full_prog+=init()
full_prog.G0(Z=-16)


full_prog+=prog
full_prog+=end()

cnc.run(full_prog)

8743.171305418015

In [16]:
init(machine=cnc).run()

^C


In [17]:
cnc.status()

'<Hold:0|MPos:-54.504,-76.100,-16.000|Bf:0,102|FS:0,0|Ov:100,100,100>'

In [18]:
cnc.reset()

(0, ['ok', '', "Grbl 1.1f ['$' for help]", 'ok'])